In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pheochromocytoma_and_Paraganglioma/GSE33371'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Beta-catenin status effects in human adrenocortical carcinomas (33), adenomas (22), and normal adrenal cortex (10)"
!Series_summary	"We scored adrenocortical carcinomas and adenomas for abnormal beta-catenin staining, and sequenced the beta-catenin gene in some samples. We compared adrenocortincal carcinomas with and without abnormal beta-catenin staining and found many significant expression differences and significant results from enrichment testing. A similar comparison in the adenomas gave relatively few differences, and they did not correlate to differences found for the carcinomas.  Abnormal beta-catenin staining was associated with mitotic rate and poorer patient survival in the carcinomas.  In a second independent data set (given in a supplement) we again found beta-catenin associated with poor survival.  The array data given is the same as GEO series GSE10927, with additional characteristics about beta-catenin, and new patient followup da

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import json

# 1. Gene Expression Data Availability
is_gene_available = True  # The dataset appears to contain mRNA data, which suggests gene expression data is available.

# 2. Variable Availability and Data Type Conversion
# Check availability and identify keys
trait_row = None  # There is no direct mention of `Pheochromocytoma_and_Paraganglioma`
age_row = 0
gender_row = 1

# 2.3 Data Type Conversion Functions

def convert_trait(value):
    return None  # No conversion needed as trait_row is None

def convert_age(value):
    try:
        value = value.split(': ')[-1]
        if value == '<10':
            return 5  # Can be set to an arbitrary low value like 5
        val = float(value)
        return val
    except Exception:
        return None

def convert_gender(value):
    try:
        value = value.split(': ')[-1]
        return 1 if value == 'M' else 0
    except Exception:
        return None

# Save Metadata
def save_cohort_info(series_id, file_path, is_gene_available, trait_avail):
    data = {
        'series_id': series_id,
        'is_gene_available': is_gene_available,
        'trait_available': trait_avail
    }
    with open(file_path, 'w') as f:
        json.dump(data, f)

save_cohort_info('GSE33371', './preprocessed/Pheochromocytoma_and_Paraganglioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (skipped if trait_row is None)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Pheochromocytoma_and_Paraganglioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/trait_data/GSE33371.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
